In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt 

tweets = pd.read_excel('ArsenalTweetsMOS_NP.xlsx', engine='openpyxl')
tweets.head(5)

,Source.Name,Column1,id,date,source,place,username,Content,hashtags,Likes
0,afcMOS_analysisV1hashtag.xlsx,0,1467278795451224064,2021-12-04 23:45:00,"<a href=""https://ads-api.twitter.com"" rel=""nof...",NaN,talkSPORT,"""When you look at Aubameyang and you look at R...","['AFC', 'MUFC']",114
1,afcMOS_analysisV1hashtag.xlsx,1,1467277137975513088,2021-12-04 23:38:25,"<a href=""http://twitter.com/download/android"" ...",NaN,by_19xx,We need to rotate between Tierney and Nuno Tav...,['afc'],0
2,afcMOS_analysisV1hashtag.xlsx,2,1467273008943443968,2021-12-04 23:22:00,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Martinel1i,Our starboys 🤩 #Afc https://t.co/OzpMPGG3v1,['Afc'],1
3,afcMOS_analysisV1hashtag.xlsx,3,1467271753726575104,2021-12-04 23:17:01,"<a href=""http://twitter.com/download/iphone"" r...",NaN,Mickeymoneychat,I LOVE this. @Arsenal get this kid to the carp...,"['AFC', 'Arsenal']",51
4,afcMOS_analysisV1hashtag.xlsx,4,1467268107311800064,2021-12-04 23:02:31,"<a href=""http://twitter.com/download/iphone"" r...",NaN,realarsenaldeal,AMN and Sambi probably deserve a start in midf...,"['afc', 'arsenal', 'EVEARS', 'arteta']",0


In [2]:
import re
from nltk.corpus import stopwords

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove whitespace
    text = text.strip()
    
    return text


In [3]:
import pandas as pd
from sklearn.utils import shuffle

# Apply the preprocessing function to the 'text' column
tweets['processed_text'] = tweets['Content'].apply(preprocess_text)

In [14]:
import torch
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the PyTorch model
model = torch.load('cnntlstm_modelSP2.pth')

# Define the label map
label_map = {0: "NEGATIVE", 1: "POSITIVE", 2: "NEUTRAL"}

# Preprocess the test data using the same tokenizer used during training
max_features = 30000
max_length = 100
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(tweets['processed_text'].values)
X_test = tokenizer.texts_to_sequences(tweets['processed_text'].values)
X_test = pad_sequences(X_test, maxlen=max_length)

# Predict the sentiment label for the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred = [label_map[label] for label in y_pred]

# Print the predicted sentiment labels
print(y_pred)


AttributeError: 'collections.OrderedDict' object has no attribute 'predict'

In [16]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Load GloVe embeddings
def load_glove_embeddings(file_path, embedding_dim, word_index, max_features):
    embeddings_index = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    
    embedding_matrix = np.zeros((max_features, embedding_dim))
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return torch.tensor(embedding_matrix, dtype=torch.float32)

# Load the GloVe embeddings matrix
glove_file_path = 'glove.twitter.27B.200d.txt'  # Update the path to the downloaded GloVe file
embedding_dim = 200
embedding_matrix = load_glove_embeddings(glove_file_path, embedding_dim, tokenizer.word_index, max_features)

class CNNTLSTM(nn.Module):
    def __init__(self, embedding_matrix, embed_dim=200, lstm_out=448, dropout_rate=0.2, num_classes=3, num_filters=256, filter_size=3, pool_size=2):
        super(CNNTLSTM, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=True)
        self.conv1d = nn.Conv1d(embed_dim, num_filters, filter_size)
        self.relu = nn.ReLU()
        self.max_pool1d = nn.MaxPool1d(pool_size)
        self.bi_lstm = nn.LSTM(num_filters, lstm_out // 2, batch_first=True, bidirectional=True, dropout=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(lstm_out, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.max_pool1d(x)
        x = x.permute(0, 2, 1)
        x, _ = self.bi_lstm(x)
        x = x[:, -1, :]
        x = self.dropout(x)
        x = self.fc(x)
        return self.softmax(x)
    


In [18]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to load a saved model
def load_saved_model(model_path):
    model = CNNTLSTM(embedding_matrix).cuda()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# # Tokenize and pad the preprocessed texts
# tokenized_texts = tokenizer.texts_to_sequences(tweets['processed_text'])
# padded_texts = pad_sequences(tokenized_texts, maxlen=100)

# # Convert the tokenized and padded texts to PyTorch tensors
 texts_torch = torch.tensor(padded_texts, dtype=torch.long)

# Load the saved model
model_path = 'cnntlstm_modelSP2.pth'
loaded_model = load_saved_model(model_path)

# Use the model to predict sentiment labels for the text data
label_map = {0: "NEGATIVE", 1: "POSITIVE", 2: "NEUTRAL"}

with torch.no_grad():
    # Move the model to the GPU
    loaded_model.cuda()
    
    # Create a DataLoader object to load the text tensors in batches
    dataset = TensorDataset(texts_torch)
    dataloader = DataLoader(dataset, batch_size=512)
    
    # Predict the sentiment labels in batches and concatenate the results
    predictions = []
    for batch in dataloader:
        batch = batch[0].cuda()
        output_probs = loaded_model(batch)
        _, batch_predictions = torch.max(output_probs.data, 1)
        batch_predictions = batch_predictions.cpu().numpy()
        batch_predictions = [label_map[pred] for pred in batch_predictions]
        predictions += batch_predictions

# Add the predicted labels to the original DataFrame and save it to a new CSV file
tweets['predicted_sentiment'] = predictions
tweets.to_csv('predictionsg2.csv', index=False)
